<a href="https://colab.research.google.com/github/Jhonattan17/LYTHPRED/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install streamlit -q

In [20]:
pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
!pip install cartopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
!apt-get -qq install -y graphviz && pip install pydot
import pydot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from google.colab import drive 

In [16]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
%%writefile app.py
# Importaciones a usar 

import streamlit as st
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
from google.colab import drive  
from sklearn.impute import KNNImputer
import cartopy
import pickle
import io
import os 

from scipy.signal import butter 
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import StratifiedKFold, cross_val_score

st.set_page_config(page_title="LYTHPRED", page_icon=":🛢️:")

# Titulo y objetivo de la app
with st.container():
  st.title('LYTHPRED :🛢️:')
  st.subheader('APLICATIVO WEB CAPAZ DE REALIZAR PREDICCIONES LITOLOGICAS DURANTE LA PERFORACION DE UN POZO PETROLERO BASADO EN INTELIGENCIA ARTIFICIAL')

# Desarrollo de la app 
with st.container():
  st.write("---")
  # Etiqueta que se mostrará 
  st.title('Etiqueta de la litologia')
  Categoria = {'Facies': ['Sandstone','Sandstone/Shale','Shale','Marl','Dolomite','limestone','Chalk','Halite','Anhydrite','Tuff','Coal','Basement'], 'Categoria':[1,2,3,4,5,6,7,8,9,10,11,12]}
  cat = pd.DataFrame(Categoria)
  st.dataframe(cat)


while True:
  with st.container():
    st.write("---")
    st.title("Visualizador de archivos LAS")
    archivo_las = st.file_uploader("Seleccione un archivo LAS", type=[".las", ".LAS"])
    if archivo_las :
      archivo_las = np.loadtxt(archivo_las, skiprows=90)
      archivo_las= np.unique(archivo_las)
      archivo_las = pd.DataFrame(archivo_las)
      # FUNCION PARA LEER TODOS LOS ARCHIVOS .LAS DE UNA CARPETA
      def leer_archivos_las(path, files):
  
        mneumonics= ['DEPTH', 'FLC', 'FLL', 'CALI', 'MW', 'ROP', 'RDEP', 'RSHA', 'RMED' ,'RXO' ,'SP' ,'DTC' ,'NPHI' ,'PEF' ,'GR' ,'RHOB' ,'DRHO' ,'DEPTH_MD' ,'X_LOC', 'Y_LOC', 'Z_LOC'] 
  
        df_concat=pd.DataFrame()

        for file in files:
          ruta=path+file
          raw_data=open(ruta)
          data=np.loadtxt(raw_data, skiprows=90)
          data = pd.DataFrame(data, columns=mneumonics)
          data= data.drop(columns=['MW','CALI', 'X_LOC', 'Y_LOC', 'Z_LOC', 'DEPTH_MD','DTC','NPHI','PEF','FLC','ROP'])
          data['WELL']=file

          df_concat=pd.concat([df_concat, data])

        return df_concat


      #FUNCION PARA IMPUTAR VALORES FALTANTES CON EL ALGORITMO KNN VECINOS.
      def imput_by_formation(loglist, df):
        imputer = KNNImputer(n_neighbors=5, weights="uniform")
        for log in loglist:
          imputer.fit(df[[log]])
          df[log] = imputer.transform(df[[log]]).ravel()
        return(df)


      #FUNCION PARA IMPUTAR VALORES FALTATES DE TODAS LAS FORMACIONES UTILIZANDO KNN VECINOS 
      def filter_by_formation(df):
        list_form = df.FLL.unique().tolist()
        loglist=['RDEP', 'RSHA', 'RMED', 'RXO', 'SP', 'GR', 'RHOB', 'DRHO']
        df_full_imputed = pd.DataFrame()
        for form in list_form:
          print("_________________________________________")
          print("\nInicio de Imputación de "+ form)
          df_filter = df[df.FLL == form]
          df_filter_imputed = imput_by_formation(loglist, df_filter)
          df_full_imputed = pd.concat([df_full_imputed, df_filter_imputed])

          print("Formacion  "+ form + " Imputada")
        return(df_full_imputed)

      path=('/content/drive/MyDrive/Force_2020_all_wells_train_test_blind_hidden_final/')
      files = ['7_1-1.las','7_1-2 S.las','15_9-13.las','15_9-14.las','15_9-23.las','16_2-11 A.las','16_5-3.las','16_7-4.las','25_2-13 T4.las','25_2-14.las','25_5-1.las','25_5-4.las','25_6-1.las','25_6-3.las','25_10-9.las','31_3-2.las','31_4-5.las','32_2-1.las','33_9-17.las']
      df_concat = pd.DataFrame()
      df_concat = leer_archivos_las(path, files)
      df_concat = df_concat.replace(-999.25, np.nan)

      #Etiqueta del programa 
      Formaciones_cot = [30000,65030,65000,80000,74000,70000,70032,88000,86000,99000,90000,93000]
      Facies= ['Sandstone','Sandstone/Shale','Shale','Marl','Dolomite','Limestone','Chalk','Halite','Anhydrite','Tuff','Coal','Basement']

      dic_Facies=dict(zip(Formaciones_cot, Facies))
      df_concat['FLL']=df_concat['FLL'].replace(dic_Facies)
      df_concat = df_concat[df_concat['FLL'].notna()]
      df_concat = df_concat.drop(df_concat[df_concat.FLL=="Anhydrite"].index)
      df_final = filter_by_formation(df_concat)

      #agregar categoria a los archivos imputados 
      Formaciones_cot_well = [1, 2, 3, 4, 5, 6, 7,8,9,10,11,12]
      Facies_well= ['Sandstone','Sandstone/Shale','Shale','Marl','Dolomite','Limestone','Chalk','Halite','Anhydrite','Tuff','Coal','Basement']
      dic_Facies_well=dict(zip( Facies_well, Formaciones_cot_well))
      df_final['FLL']=df_final['FLL'].replace(dic_Facies_well)
      df_final = df_final[df_final['FLL'].notna()]   

      filename = st.text_input('Ingrese el nombre completo del archivo .las')
      if filename:      
        df_well_prueba= df_final.loc[df_final['WELL'] == filename]
        st.dataframe(df_well_prueba)
      
        litologia = df_well_prueba.FLL.unique()
        lista_litofacies_pozo = litologia.tolist()
        df_lista_resultado_litologia = pd.DataFrame({'LITOLOGY': lista_litofacies_pozo})
        st.subheader("LITOFACIES DEL POZO EVALUADO")
        st.dataframe(df_lista_resultado_litologia)



        # codigo del modelo
        X_train = df_well_prueba[['RDEP', 'RSHA', 'RMED', 'RXO', 'SP', 'GR', 'RHOB', 'DRHO']]
        y_train = df_well_prueba.FLL
        scaler = StandardScaler()
        pipe = make_pipeline(scaler, KNN(n_neighbors=5))
        scaler.fit(X_train)
        pipe.fit(X_train, y_train)  # Añadir esta línea para ajustar el modelo
        cv = StratifiedKFold(5, shuffle=True)
        cv_scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring='accuracy')
        mean_cv_scores = np.mean(cv_scores)
        y_pred = pipe.predict(X_train)
        lista_y_pred = y_pred.tolist()
        df_lista_resultado = pd.DataFrame({'DEPTH': df_well_prueba['DEPTH'], 'FLLPD': lista_y_pred})

      
        # merge que une el dataframe real con la prediccion
        df_resultado = pd.merge(df_well_prueba, df_lista_resultado, on="DEPTH")
        st.subheader("DATAFRAME DEL ARCHIVO .LAS DEL POZO EVALUADO CON LA COLUMNA CORRESPONDIENTE DE LA PREDICCIÒN")
        st.dataframe(df_resultado)

        # grafico real + prediccion del modelo
        f=df_resultado.FLL.values
        depth_prediction = df_resultado.DEPTH.values
        facies_prediction=[]
        for i  in range(len(depth_prediction)):
          facies_prediction.append(f[i])
        df_resultado['FLL']=facies_prediction


        f=df_resultado.FLLPD.values
        depth_prediction = df_resultado.DEPTH.values
        facies_prediction_knn=[]
        for i  in range(len(depth_prediction)):
          facies_prediction_knn.append(f[i])
        df_resultado['FLLPD']=facies_prediction_knn

        # codigo del grafico 
        def log_plot(df_resultado):
          df_resultado= df_resultado.sort_values(by='DEPTH')

          rows,cols = 1,10
          fig,ax= plt.subplots(nrows=rows, ncols=cols, figsize=(12,6), sharey=True)
          ax[0].plot(df_resultado.RDEP, df_resultado.DEPTH, color='black')
          ax[0].set_title('RDEP')
          ax[0].minorticks_on()
          ax[0].grid(which='major', linestyle='-',linewidth='0.5', color='lime')
          ax[0].grid(which='minor', linestyle=':',linewidth='0.5', color='black')
          ax[1].plot(df_resultado.RSHA, df_resultado.DEPTH, color='black')
          ax[1].set_title('RSHA')
          ax[1].minorticks_on()
          ax[1].grid(which='major', linestyle='-',linewidth='0.5', color='lime')
          ax[1].grid(which='minor', linestyle=':',linewidth='0.5', color='black')
          ax[2].plot(df_resultado.RMED, df_resultado.DEPTH, color='black')
          ax[2].set_title('RMED')
          ax[2].minorticks_on()
          ax[2].grid(which='major', linestyle='-',linewidth='0.5', color='lime')
          ax[2].grid(which='minor', linestyle=':',linewidth='0.5', color='black')
          ax[3].plot(df_resultado.RXO, df_resultado.DEPTH, color='black')
          ax[3].set_title('RXO')
          ax[3].minorticks_on()
          ax[3].grid(which='major', linestyle='-',linewidth='0.5', color='lime')
          ax[3].grid(which='minor', linestyle=':',linewidth='0.5', color='black')
          ax[4].plot(df_resultado.SP, df_resultado.DEPTH, color='black')
          ax[4].set_title('SP')
          ax[4].minorticks_on()
          ax[4].grid(which='major', linestyle='-',linewidth='0.5', color='lime')
          ax[4].grid(which='minor', linestyle=':',linewidth='0.5', color='black')
          ax[5].plot(df_resultado.GR, df_resultado.DEPTH, color='black')
          ax[5].set_title('GR')
          ax[5].minorticks_on()
          ax[5].grid(which='major', linestyle='-',linewidth='0.5', color='lime')
          ax[5].grid(which='minor', linestyle=':',linewidth='0.5', color='black')
          ax[6].plot(df_resultado.RHOB, df_resultado.DEPTH, color='black')
          ax[6].set_title('RHOB')
          ax[6].minorticks_on()
          ax[6].grid(which='major', linestyle='-',linewidth='0.5', color='lime')
          ax[6].grid(which='minor', linestyle=':',linewidth='0.5', color='black')
          ax[7].plot(df_resultado.DRHO, df_resultado.DEPTH, color='black')
          ax[7].set_title('DRHO')
          ax[7].minorticks_on()
          ax[7].grid(which='major', linestyle='-',linewidth='0.5', color='lime')
          ax[7].grid(which='minor', linestyle=':',linewidth='0.5', color='black')


          for i in range (len(ax)):
            ax[i].set_ylim(max(df_resultado.DEPTH), min(df_resultado.DEPTH))
 

          F = np.vstack((facies_prediction, facies_prediction)).T
          ax[8].imshow(F, aspect='auto', extent=[0,1, max(df_resultado.DEPTH), min(df_resultado.DEPTH)])
          ax[8].set_title('TRUE')

          F= np.vstack((facies_prediction_knn, facies_prediction_knn)).T
          ax[9].imshow(F, aspect='auto', extent=[0,1, max(df_resultado.DEPTH), min(df_resultado.DEPTH)])
          ax[9].set_title('PRED_FACIES')

          plt.suptitle('WELL TRUE-PREDICTION FACIES KNN', size=15)
        st.subheader("GRAFICO DE REGISTROS Y COMPARACION DE LA LITOLOGIA REAL Y LA PREDICCION UTILIZANDO KNN")
        fig = log_plot(df_resultado)
        st.set_option('deprecation.showPyplotGlobalUse', False)
        st.pyplot(fig)
  hacer_otra_prediccion = st.button("REALIZAR OTRA PREDICCION")
  if not hacer_otra_prediccion:
    break


Writing app.py


In [ ]:
! streamlit run  app.py & npx localtunnel --port 8501



npx: installed 22 in 4.257s
your url is: https://green-pumas-sell-35-229-55-113.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.55.113:8501

_________________________________________

Inicio de Imputación de Sandstone
/content/app.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[log] = imputer.transform(df[[log]]).ravel()
/content/app.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[log] = imputer.transform